In [1]:
from altProts_in_communities.utils import *
from altProts_in_communities.compute_features import *

In [2]:
bioplex_psms = pickle.load(open('OP16_bioplex_all_psms.pkl', 'rb'))

In [3]:
peptide_counts_dict = {}
for bait, exps in bioplex_psms.items():
    peptide_counts_dict[bait] = {}
    for exp_id, psms in exps['exps'].items():
        for prey, psm_dict in psms['psms'].items():
            cnt_pep = len(set(x[0] for x in psm_dict['psms']))
            cnt_upep = psm_dict['n_upep']
            if prey not in peptide_counts_dict[bait]:
                peptide_counts_dict[bait][prey] = {
                    'cnt_pep':cnt_pep,
                    'cnt_upep':cnt_upep, 
                }
            else:
                peptide_counts_dict[bait][prey]['cnt_pep']  += cnt_pep
                peptide_counts_dict[bait][prey]['cnt_upep'] += cnt_upep

In [4]:
min_entropy = 0.5
cnt_low_ent  = 0
dtypes = {'Experiment.ID':int, 'Bait':sanitize, 'Prey':sanitize, 'AvePSM':float, 'Z':float, 'WD':float, 'Entropy':float}
comppass_out = []
with open('BioPlex_comppass_output.csv', 'r') as f:
    for n,l in enumerate(f):
        fix_commas = {txt_field:txt_field.replace(',', '@') for txt_field in re.findall(r'"([^"]*)"', l) if ',' in txt_field}
        for txt_field in fix_commas:
            l = l.replace(txt_field, fix_commas[txt_field])
        ls = l.strip().split(',')
        if n==0:
            keys = [sanitize(x) for x in ls]
            continue
        line = dict(zip(keys, ls))
        for i in line:
            line[i] = dtypes[i](line[i])
            if i in {'Bait', 'Prey'}:
                line[i] = line[i].replace('@', ',')
        if line['Entropy'] < min_entropy:
            cnt_low_ent += 1
            continue
        bait, prey = line['Bait'], line['Prey']
        line.update({
            'cnt_upeps': peptide_counts_dict[bait][prey]['cnt_upep'],
            'cnt_peps':  peptide_counts_dict[bait][prey]['cnt_pep'],
            'Prey_Gene': 'unknown' if line['Prey'] not in prot_gene_dict else prot_gene_dict[line['Prey']],
            'Batch':     bait_batch_dict[line['Bait']],
            'is_alt':    is_alt(line['Prey']),
            'excluded_by_binom_test': False
        })
        comppass_out.append(line)
        
print('eliminated by entropy filter:', cnt_low_ent)

eliminated by entropy filter: 350279


In [5]:
comppass_out_df = pd.DataFrame(comppass_out)
comppass_out_df.to_csv('comppass_out_df.csv')
comppass_out_df.head()
print('Total number of baits', len(comppass_out_df.Bait.unique()))

Total number of baits 3005


In [6]:
preys              = comppass_out_df.Prey.unique()
print('Total number of preys indentified:',len(preys))
print('Number of alt preys indentified:', len(set([x for x in preys if is_alt(x)])))
tot_ips            = sum(v for k,v in batch_n_wells.items())
batch_prey_ip      = comppass_out_df.groupby(['Prey', 'Batch']).Bait.nunique()
prey_tot_ip        = comppass_out_df.groupby('Prey').Bait.nunique().to_dict()
prey_batches       = comppass_out_df.groupby('Prey').Batch.unique().to_dict()
prey_batch         = comppass_out_df.groupby(['Prey', 'Batch']).count().reset_index()[['Prey', 'Batch']].values
prey_batch_freq    = {(p, b) : batch_prey_ip[p][b] / batch_n_wells[b] for p,b in prey_batch}
prey_tot_freq      = {p : prey_tot_ip[p] / tot_ips for p in preys}
bait_batch_ave_psm = comppass_out_df.groupby(['Batch', 'Bait']).AvePSM.mean().to_dict()

Total number of preys indentified: 8322
Number of alt preys indentified: 262


In [7]:
# pass all preys through binomial filter, adjusting p values to 1% fdr with Bejanimi Hochenberg
batch_enriched_preys = []
for n,prey in enumerate(preys):
    if n>100:break
    p_vals = []
    p = prey_tot_freq[prey]
    for batch in prey_batches[prey]:
        p_val = binom_test(batch_prey_ip[prey][batch], batch_n_wells[batch], p, alternative='greater')
        p_vals.append(p_val)
    bh = sm.stats.multipletests(p_vals, alpha = 0.001, method='fdr_bh')
    binom_test_res = list(zip(prey_batches[prey], [prey]*len(prey_batches[prey]),  bh[0]))
    batch_enriched_preys += [(x[0], x[1]) for x in binom_test_res if x[2]]

bep_set = set(batch_enriched_preys)
len(bep_set) / len(comppass_out_df)

0.0029132252776136493

In [8]:
comppass_out_df = comppass_out_df.set_index(['Batch', 'Prey'])

In [9]:
binom_test_exlusion = set()
with tqdm(total=len(bep_set), file=sys.stdout) as pbar:
    i=0
    for bep in list(bep_set):
        batch, prey = bep
        batch_prey_baits_avepsms = comppass_out_df.loc[bep, ['Bait', 'AvePSM']].values
        for bait_prey in batch_prey_baits_avepsms:
            bait, avepsm = bait_prey
            if  avepsm < bait_batch_ave_psm[(batch, bait)] :
                comppass_out_df.loc[bep, 'excluded_by_binom_test'] = True
        i+=1
        pbar.set_description('processed: %d' % (1 + i))
        pbar.update(1)

processed: 2:   0%|          | 1/2056 [00:00<05:46,  5.93it/s]

/home/sleblanc/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:873: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
/home/sleblanc/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2877: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)
/home/sleblanc/anaconda3/lib/python3.7/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


processed: 2057: 100%|██████████| 2056/2056 [05:13<00:00,  6.55it/s]


In [10]:
comppass_out_df = comppass_out_df.reset_index()

comppass_out_df = comppass_out_df[~comppass_out_df['excluded_by_binom_test']]

n_baits_by_prey = dict(comppass_out_df.groupby('Prey').Bait.nunique())
prey_tot_psm = dict(comppass_out_df.groupby('Prey').AvePSM.sum())
n_baits = comppass_out_df.Bait.nunique()

comppass_out_df['Ratio']            = comppass_out_df.apply(lambda x: n_baits_by_prey[x['Prey']]/n_baits, axis=1)
comppass_out_df['total_PSMs']       = comppass_out_df.apply(lambda x: prey_tot_psm[x['Prey']], axis=1)
comppass_out_df['ratio_total_PSMs'] = comppass_out_df.apply(lambda x: prey_tot_psm[x['Prey']]/x['AvePSM'], axis=1)
comppass_out_df['pep_bin']          = comppass_out_df.apply(lambda x: x['cnt_upeps'] if x['cnt_upeps']<10 else 10, axis=1)
comppass_out_df['pep_ratio']        = comppass_out_df.apply(lambda x: x['cnt_upeps'] / x['cnt_peps'], axis=1)

batch_prey_psms = comppass_out_df.groupby(['Batch', 'Prey']).AvePSM.apply(list)
prey_batches    = comppass_out_df.groupby('Prey').Batch.apply(list)

In [11]:
def compute_batch_dist(prey):
    batch_prey_psm_dist = {}
    for batch in prey_batches[prey]:
        psm_ls = batch_prey_psms[batch][prey]
        if all(x == psm_ls[0] for x in psm_ls): 
            psm_ls[0] += 0.5
        psms = psm_ls + [0]*(batch_n_wells[batch] - len(psm_ls))
        batch_prey_psm_dist[(batch, prey)] = (np.mean(psms), np.std(psms))
    return batch_prey_psm_dist

In [12]:
preys = comppass_out_df.Prey.unique()
batch_prey_psm_dists = []
with Pool(8) as p:
    batch_prey_psm_dists.append(p.map(compute_batch_dist, preys))

batch_prey_psm_dist = {}
for p in batch_prey_psm_dists[0]:
    batch_prey_psm_dist.update(p)

In [13]:
# bin features in 1000 equally sized bins
comppass_out_df['batch_Z'] = comppass_out_df.apply(lambda x: batch_Z(x['Batch'], x['Prey'], x['AvePSM'], batch_prey_psm_dist), axis=1)
for feat in feats:
    comppass_out_df[feat+'_binned'] = pd.cut(comppass_out_df[feat], bin_feature(comppass_out_df[feat]), labels=False, retbins=True, right=True)[0]

In [14]:
len([x for x in comppass_out_df.Prey.unique() if is_alt(x)])

262

In [15]:
!cp BP2_OP16_features.csv BP2_OP16_features.csv.back

In [16]:
comppass_out_df.to_csv('BP2_OP16_features.csv')